### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
!pip install tensorflow tensorflow-hub
!pip install opencv-python moviepy


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 10.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 59.9 MB/s eta 0:00:00:00:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110721 sha256=1aaafc93f87e2f1a43eaac95e505784026c29d443b18ab9ccb9f5ac62050fc77
  Stored in directory: /root/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical


In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

# Data Generator for batch processing
def video_data_generator(video_paths, labels, batch_size=8, max_frames=64, frame_size=(224, 224)):
    while True:
        for start in range(0, len(video_paths), batch_size):
            end = min(start + batch_size, len(video_paths))
            batch_videos = []
            batch_labels = []
            
            for i in range(start, end):
                cap = cv2.VideoCapture(video_paths[i])
                frames = []
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    # Resize and normalize the frame
                    frame = cv2.resize(frame, frame_size)
                    frame = frame / 255.0
                    frames.append(frame)
                    
                    if len(frames) == max_frames:
                        break

                cap.release()

                # Ensure all videos have the same number of frames
                if len(frames) < max_frames:
                    for _ in range(max_frames - len(frames)):
                        frames.append(np.zeros((224, 224, 3)))  # padding

                batch_videos.append(np.array(frames))
                batch_labels.append(labels[i])

            yield np.array(batch_videos), np.array(batch_labels)

# Paths for your video dataset
fight_video_paths = ['/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/' + f for f in os.listdir('/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/')]
nofight_video_paths = ['/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/' + f for f in os.listdir('/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/')]

# Labels: 1 for fight, 0 for no fight
fight_labels = [1] * len(fight_video_paths)
nofight_labels = [0] * len(nofight_video_paths)

# Combine paths and labels
video_paths = fight_video_paths + nofight_video_paths
labels = fight_labels + nofight_labels

# Train-test split
train_video_paths, test_video_paths, train_labels, test_labels = train_test_split(
    video_paths, labels, test_size=0.2, random_state=42
)

# Convert labels to categorical (one-hot encoding)
train_labels = to_categorical(train_labels, num_classes=2)
test_labels = to_categorical(test_labels, num_classes=2)

# Batch size
batch_size = 8

# Create data generators for training and testing
train_gen = video_data_generator(train_video_paths, train_labels, batch_size=batch_size)
test_gen = video_data_generator(test_video_paths, test_labels, batch_size=batch_size)

# Model setup (example)
model = models.Sequential([
    layers.Input(shape=(64, 224, 224, 3)),  # 64 frames, 224x224, 3 channels (RGB)
    layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.GlobalAveragePooling3D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # 2 classes: fight, no-fight
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
steps_per_epoch = len(train_video_paths) // batch_size
validation_steps = len(test_video_paths) // batch_size

model.fit(train_gen, epochs=50, steps_per_epoch=steps_per_epoch, validation_data=test_gen, validation_steps=validation_steps)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_gen, steps=validation_steps)
print(f"Test accuracy: {test_accuracy}")
# Save the trained model
model.save('/kaggle/working/fight_detection_model.h5')  # Change the path and filename as needed


Epoch 1/50


I0000 00:00:1726641048.190282     117 service.cc:145] XLA service 0x7f8c40006a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726641048.190345     117 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-09-18 06:30:52.435794: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[8,32,64,224,224]{4,3,2,1,0}, u8[0]{0}) custom-call(f32[8,3,64,224,224]{4,3,2,1,0}, f32[32,3,3,3,3]{4,3,2,1,0}, f32[32]{0}), window={size=3x3x3 pad=1_1x1_1x1_1}, dim_labels=bf012_oi012->bf012, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-18 06:30:53.289652: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.85404

200/200 ━━━━━━━━━━━━━━━━━━━━ 426s 2s/step - accuracy: 0.5021 - loss: 0.6964 - val_accuracy: 0.7000 - val_loss: 0.6548
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 381s 2s/step - accuracy: 0.6185 - loss: 0.6407 - val_accuracy: 0.7450 - val_loss: 0.5468
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 380s 2s/step - accuracy: 0.7316 - loss: 0.5429 - val_accuracy: 0.7450 - val_loss: 0.5099
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 379s 2s/step - accuracy: 0.7568 - loss: 0.4961 - val_accuracy: 0.7625 - val_loss: 0.4780
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 364s 2s/step - accuracy: 0.7775 - loss: 0.4650 - val_accuracy: 0.7700 - val_loss: 0.4577
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 366s 2s/step - accuracy: 0.7985 - loss: 0.4435 - val_accuracy: 0.7750 - val_loss: 0.4482
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 367s 2s/step - accuracy: 0.7975 - loss: 0.4388 - val_accuracy: 0.7575 - val_loss: 0.4422
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 364s 2s/step - accuracy: 0.8047 - loss: 0.4315 - val_accuracy: 0.780

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained 3D CNN model
model = load_model('/kaggle/input/fight-detection-3d-cnn/pytorch/default/1/fight_detection_model (3).h5')  # Replace with your model's path on Kaggle

# Function to test the model on a video and overlay "FIGHT" text when violence is detected
def test_on_video(input_video_path, output_video_path):
    # Open the input video
    video = cv2.VideoCapture(input_video_path)

    # Get video properties
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))

    # Video writer to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Sliding window to collect frames
    sliding_window = []
    max_frames = 64  # Matching the input shape of the model

    while video.isOpened():
        ret, frame = video.read()

        if not ret:
            break  # Stop if video ends

        # Preprocess the frame (resize and normalize)
        resized_frame = cv2.resize(frame, (224, 224))  # Resize frame to match the input size
        resized_frame = resized_frame / 255.0  # Normalize pixel values to [0,1]
        sliding_window.append(resized_frame)

        # If we have enough frames for a prediction (e.g., 64 frames for 3D CNN)
        if len(sliding_window) == max_frames:
            # Prepare the batch of frames for the model
            input_frames = np.array(sliding_window)
            input_frames = np.expand_dims(input_frames, axis=0)  # Add batch dimension
            input_frames = input_frames.astype('float32')

            # Make prediction
            prediction = model.predict(input_frames)
            predicted_class = np.argmax(prediction, axis=1)[0]  # 0: No Fight, 1: Fight

            # If "Violence" (fight) is detected, overlay red "FIGHT" text at the lower center
            if predicted_class == 1:  # Violence class
                # Get text size and calculate lower center position
                text = "FIGHT"
                font_scale = 1  # Reduced font size
                font_thickness = 1  # Reduced boldness
                font = cv2.FONT_HERSHEY_SIMPLEX
                text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
                
                # Set the position for the text
                text_x = (frame_width - text_size[0]) // 2  # Center horizontally
                text_y = frame_height - 30  # Closer to bottom (30 pixels from the bottom)
                
                # Define the rectangle parameters with reduced size
                rect_width = text_size[0] + 10  # Smaller width of the rectangle with padding
                rect_height = text_size[1] + 10  # Smaller height of the rectangle with padding
                rect_x = text_x - 5  # Rectangle's top-left x position
                rect_y = text_y - text_size[1] - 5  # Rectangle's top-left y position
                
                # Draw a transparent rectangle with a red outline
                overlay = frame.copy()  # Make a copy of the frame to draw on
                alpha = 0.1  # Transparency factor
                cv2.rectangle(overlay, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), (0, 0, 255), -1)  # Red filled rectangle
                cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)  # Add transparency

                # Draw the red outline for the rectangle
                cv2.rectangle(frame, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), (0, 0, 255), 1)  # Thinner red outline
                
                # Overlay the red text with anti-aliasing
                cv2.putText(frame, text, (text_x, text_y), font, font_scale, (0, 0, 255), font_thickness, cv2.LINE_AA)

            # Write the frame to the output video
            out.write(frame)

            # Remove the oldest frame from the sliding window (for real-time processing)
            sliding_window.pop(0)

    # Release the video objects
    video.release()
    out.release()

    print(f"Test completed. The video is saved at {output_video_path}")

# Example usage
input_video_path = '/kaggle/input/test-video/fight video demo.mp4'  # Replace with your input video path on Kaggle
output_video_path = '/kaggle/working/output_video_3DCNN2.mp4'  # Path to save the output video on Kaggle

test_on_video(input_video_path, output_video_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms